**Задание**: LLM классификация отзывов

**Метрика**: Weighted F1

**Time**: 5 секунд

# 1. Подготовка данных: text preprocessing

Анализируем train тексты

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv('train.csv')['text'].tolist()
data_train[:10]  # пример нескольких первых отзывов

Очистка и обработка текстов

In [ ]:
# приводим к нижнему регистру
data_train = [i.lower() for i in data_train]

In [ ]:
# удалим стоп-слова, HTML-тэги, цифры и знаки препинания
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
import nltk

nltk.download("stopwords")
stop_words = set(stopwords.words("russian"))

In [ ]:
def clean_text(text):
    # убираем HTML тэги
    text = BeautifulSoup(text, "lxml").text
    # убираем цифры и спецсимволы, оставляем только буквы и пробелы
    text = re.sub(r"[^а-яА-Яa-zA-Z\s]", " ", text)
    # убираем лишние пробелы
    text = re.sub(r"\s+", " ", text).strip()
    # удаляем стоп-слова
    tokens = text.split()
    tokens = [w for w in tokens if w not in stop_words]
    text = ' '.join(tokens)
    return text

In [ ]:
data_train = [clean_text(i) for i in data_train]

In [ ]:
data_train[:5]

Датасет небольшой, здесь сделай аугментацию данных для получения большего количества обучающих примеров - через Back Translation

Back translation (ru - eng - ru)

In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline

# Может потребовать установки моделей
translator_en_ru = pipeline("translation", model="Helsinki-NLP/opus-mt-en-ru")
translator_ru_en = pipeline("translation", model="Helsinki-NLP/opus-mt-ru-en")

def back_translation(text):
    # Русский -> Английский -> Русский
    en_text = translator_ru_en(text)[0]['translation_text']
    ru_text = translator_en_ru(en_text)[0]['translation_text']
    return ru_text

In [ ]:
data_train_ru_eng = []

In [ ]:
for i in data_train:
  data_train_ru_eng.append(back_translation(i))

In [ ]:
data_train[:5]

In [ ]:
data_train_ru_eng[:5]

In [ ]:
data_train_ru_eng = [clean_text(i) for i in data_train_ru_eng]

In [ ]:
len(data_train_ru_eng)

In [ ]:
for i in data_train_ru_eng:
  data_train.append(i)

In [ ]:
len(data_train)

Проведём **лемматизацию**: все слова приведём к начальной форме

In [ ]:
!pip install natasha

In [ ]:
from natasha import Doc, Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger

segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)

def lemmatization(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    return " ".join([t.lemma for t in doc.tokens])

In [ ]:
data_train = [lemmatization(i) for i in data_train]

In [ ]:
data_train = [clean_text(text) for text in data_train]

1. Построим гистограмму длин отзывов по количеству слов

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# строим гистограмму по количеству слов в отзыве
plt.hist([len(i.split()) for i in data_train], bins=50)
plt.show()

In [ ]:
# посчитаем процент отзывов с разным количеством слов
small = len([i for i in data_train if len(i.split()) <= 2])*100 / len(data_train)

In [ ]:
medium = len([i for i in data_train if len(i.split()) < 20 and len(i.split()) > 2])*100 / len(data_train)

In [ ]:
large = len([i for i in data_train if len(i.split()) >= 20])*100 / len(data_train)

In [ ]:
print(f'Процент отзывов, где менее 3 слов: {small}')
print(f'Процент отзывов, где от 3 до 19 слов: {medium}')
print(f'Процент отзывов, где более 19 слов: {large}')

Количество коротких отзывов около 6%, это не значительно

In [ ]:
len(data_train)

2. Построим WordCloud - проанализируем частотность слов

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
all_train_text = " ".join(data_train)

wc = WordCloud(width=500, height=300, background_color="white",
               colormap="viridis", max_words=100).generate(all_train_text)

plt.figure(figsize=(12, 6))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.show()

# 2. Auto-labeling

Zero-shot классификация

In [ ]:
import torch

In [ ]:
# модель
classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=0 if torch.cuda.is_available() else -1
)

In [ ]:
category = ['бытовая техника', 'обувь', 'одежда', 'посуда', 'текстиль', 'товары для детей', 'украшения и аксессуары', 'электроника', 'нет товара']

In [ ]:
def zero_shot_labeling(texts, labels, confidence_threshold=0.7):
    results = []

    for i, text in enumerate(texts):
        if i % 100 == 0:
            print(f"Обработано {i}/{len(texts)} текстов...")

        result = classifier(text, labels, multi_label=False)

        predicted_label = result['labels'][0]
        confidence = result['scores'][0]

        results.append({
            'text': text,
            'predicted_label': predicted_label,
            'confidence': confidence,
            'keep_for_training': confidence >= confidence_threshold
        })

    return pd.DataFrame(results)

In [ ]:
# убираем пустые тексты
data_train = [i for i in data_train if len(i) > 0]

In [ ]:
len(data_train)

In [ ]:
auto_labeled_df = zero_shot_labeling(data_train, category)

In [ ]:
auto_labeled_df.head()

Получили размеченный датасет с признаками confidence и keep_for_training. Для дальнейшего дообучения модели оставим только те, что keep_for_training=True

In [ ]:
qual_data_train = auto_labeled_df[auto_labeled_df['keep_for_training'] == True]

In [ ]:
# остаётся совсем не много качественный примеров
qual_data_train.shape

In [ ]:
# распределение уверенности
plt.figure(figsize=(10, 6))
auto_labeled_df['confidence'].hist(bins=20)
plt.title('Распределение уверенности модели')
plt.xlabel('Уверенность')
plt.ylabel('Количество примеров')
plt.axvline(x=0.7, color='r', linestyle='--', label='Порог 0.7')
plt.legend()
plt.show()

# Статистика по категориям
category_stats = auto_labeled_df.groupby('predicted_label').agg({
    'confidence': 'mean',
    'keep_for_training': 'sum'
}).sort_values('keep_for_training', ascending=False)

print("Статистика по категориям:")
print(category_stats)

Для электроники, обуви и нет товара нет (или почти нет) хороших обучающих примеров. Возьмём хотя бы несколько с самыми высокими confidence.

In [ ]:
rare_categories = ["электроника", "обувь", "нет товара"]

min_examples_per_category = 10  # Минимум примеров на категорию

supplemental_data = []

for category in rare_categories:
    # Берем все примеры этой категории
    category_examples = auto_labeled_df[auto_labeled_df['predicted_label'] == category]

    if len(category_examples) > 0:
        # Сортируем по уверенности (от высокой к низкой)
        sorted_examples = category_examples.sort_values('confidence', ascending=False)

        # Берем топ-N примеров
        n_to_take = min(min_examples_per_category, len(sorted_examples))
        top_examples = sorted_examples.head(n_to_take)

        print(f"Категория '{category}': взято {n_to_take} лучших примеров")
        supplemental_data.append(top_examples)
    else:
        print(f"Категория '{category}': нет примеров вообще")

# Объединяем с основными качественными данными
if supplemental_data:
    supplemental_df = pd.concat(supplemental_data)
    # ПОНИЖАЕМ ПОРОГ ДЛЯ ЭТИХ КАТЕГОРИЙ - помечаем как годные для обучения
    supplemental_df['keep_for_training'] = True

    # Объединяем с основными данными
    final_training_data = pd.concat([
        qual_data_train,  # исходные качественные данные
        supplemental_df     # дополнение для редких категорий
    ])
else:
    final_training_data = qual_data_train

In [ ]:
final_training_data.shape

In [ ]:
# сохраним все предсказания
final_training_data.to_csv('final_training_data.csv', index=False)

In [ ]:
final_training_data.head()

**Получили размеченный датасет**



In [ ]:
X = final_training_data['text'].tolist()

In [ ]:
category_ind = {'бытовая техника': 0, 'обувь': 1, 'одежда': 2,
                'посуда': 3, 'текстиль': 4, 'товары для детей': 5,
                'украшения и аксессуары': 6, 'электроника': 7, 'нет товара': 8}

In [ ]:
pred_labels = final_training_data['predicted_label'].tolist()

In [ ]:
y = [category_ind[label] for label in pred_labels]

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(X, y, test_size=0.33, random_state=42)

# 3. Дообучение модели с помощью LoRA и получение предсказание на test, в том числе среднее время обработки

In [ ]:
!pip install transformers datasets accelerate peft

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model
import numpy as np

In [ ]:
# базовая модель и токенизатор
model_name = "cointegrated/rubert-tiny"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=9)

In [ ]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "key", "value", "dense"], # адаптируем только attention-слои
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS"
)
model = get_peft_model(base_model, lora_config)

In [ ]:
enc_train = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
enc_val = tokenizer(val_texts, truncation=True, padding=True, max_length=128)

In [ ]:
train_ds = Dataset.from_dict({**enc_train, "labels": train_labels})
val_ds = Dataset.from_dict({**enc_val, "labels": val_labels})

In [ ]:
from sklearn.metrics import f1_score
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return {"f1_weighted": f1_score(labels, preds, average="weighted")}

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    num_train_epochs=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    metric_for_best_model="f1_weighted",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Метрика вышла на плато -> останавливаю обучение

In [ ]:
from transformers import AutoModelForSequenceClassification

best_model = AutoModelForSequenceClassification.from_pretrained(
    "best_rubert_model",
    num_labels=9,  # ← КРИТИЧЕСКИ ВАЖНО: укажите 9 классов!
    ignore_mismatched_sizes=True  # игнорировать несовпадение размеров
)

Посмотрим на sample submission

In [ ]:
sample = pd.read_csv('submission_example.csv')
sample.head(3)

Загружаем test.csv

In [ ]:
test_df = pd.read_csv('test.csv')
test_texts = test_df['text'].tolist()

In [ ]:
test_df.shape

In [ ]:
import time

In [ ]:
def predict_categories(texts, model, tokenizer, label_map):  # с замером среднего времени обработки
    predictions = []
    work_time = []
    for i, text in enumerate(texts):
        begin_time = time.time()
        if i % 100 == 0:
            print(f"Обработано {i}/{len(texts)}...")

        # Токенизация
        inputs = tokenizer(
            text,
            return_tensors="pt",
            truncation=True,
            max_length=512,
            padding=True
        )

        # Предсказание
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class = torch.argmax(logits, dim=-1).item()

        # Преобразование в текстовую метку
        predicted_label = label_map[predicted_class]
        predictions.append(predicted_label)
        end_time = time.time()
        work_time.append(end_time - begin_time)

    return predictions, work_time

In [ ]:
category_ind  # надо сделать обратный

In [ ]:
category_ind_back = {
    0: 'бытовая техника', 1: 'обувь', 2: 'одежда',
    3: 'посуда', 4: 'текстиль', 5: 'товары для детей',
    6: 'украшения и аксессуары', 7: 'электроника', 8: 'нет товара'
}

In [ ]:
predicted_labels = predict_categories(test_texts, best_model, tokenizer, category_ind_back)

In [ ]:
submission_df = pd.DataFrame({
    'category': predicted_labels[0]
})

submission_df.to_csv('my_submission1.csv', index=False)

In [ ]:
time_preprocess = np.array(predicted_labels[1]).mean()

In [ ]:
print(f'Среднее время обработки 1 текста в тестовой выборке = {time_preprocess}')